In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
exp_phase_kinetics.py
~~~~~~~~~~~~~~~~~~~~~
Two-point kinetic & stoichiometric calculations for a **user-defined batch
window** (typically the exponential phase) of CHO fed-batch cultures.

How it works
------------
* The user supplies `t_start` & `t_end` (h).  
* For each Clone × Rep we pick **the first (t₀)** and **last (t₁)** samples
  inside that window and treat it like a classic two-point batch culture:
    • μ from VCD₀, VCD₁  
    • **IVCD_int**  = ∫ VCD dt  (cells·h·mL⁻¹)  
    • **IVCC_int**  = IVCD_int × V̅   (cells·h)  
    • ΔX, ΔG, ΔL, ΔGln, ΔGlu, ΔrP in total moles/grams (C·V)  
    • Yields and specific rates (q’s use IVCC_int).

Inputs
------
`data/data.csv`  
Columns required: Clone, Rep, t_hr, VCD, **Vol_mL**, Glc_g_L, Lac_g_L,
Gln_mM, Glu_mM, rP_mg_L

Outputs
-------
• `outputs/kinetics_by_clone_rep.csv` – one row per Clone × Rep  
• `outputs/kinetics_by_clone.csv`     – mean ± SD per Clone

New columns
-----------
mu, IVCD, IVCC, dX, dG, dL, dQln, dQlu, dP,
Y_XG, Y_XL, Y_XQln, Y_XQlu,
q_G, q_L, q_Gln, q_Glu, q_rP

Author
------
Emiliano Balderas R. | 16 Jul 2025  
Last edit : 08 Aug 2025 — aligned with interval_kinetics.py
"""

from pathlib import Path
import numpy as np
import pandas as pd

# ───── Configuration ───────────────────────────────────────────────── #
DATA_FILE  = Path("data/data.csv")
OUT_REP    = Path("outputs/kinetics_by_clone_rep.csv")
OUT_CLONE  = Path("outputs/kinetics_by_clone.csv")

MM_GLUCOSE = 180.156   # g·mol⁻¹
MM_LACTATE =  90.080   # g·mol⁻¹
PG_PER_G   = 1e12      # g → pg

# ───── Ask user for the batch window (if run as script) ─────────────── #
def get_phase_window() -> tuple[float, float]:
    try:
        t0 = float(input("Start of exponential phase (h): "))
        t1 = float(input("End   of exponential phase (h): "))
        if t0 >= t1:
            raise ValueError
    except Exception:
        print("⚠️  Invalid input → defaulting to 0–96 h")
        t0, t1 = 0.0, 96.0
    return t0, t1

if __name__ == "__main__":
    EXP_START, EXP_END = get_phase_window()
else:                                # when imported as a module
    EXP_START, EXP_END = 0.0, 96.0

# ───── 1) Load & pre-process data (same pattern as interval_kinetics) ── #
if not DATA_FILE.exists():
    raise FileNotFoundError(f"❌ Input file not found:\n  {DATA_FILE}")

df = pd.read_csv(DATA_FILE, skiprows=1)

# — tidy dtypes —
df["t_hr"]  = pd.to_numeric(df["t_hr"], errors="coerce")
df["Rep"]   = pd.Categorical(
    pd.to_numeric(df["Rep"], errors="coerce"),
    categories=[1, 2, 3],
    ordered=True,
)
df["Clone"] = df["Clone"].astype("category")

for col in ["Vol_mL", "Glc_g_L", "Lac_g_L", "Gln_mM", "Glu_mM", "rP_mg_L"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = (
    df.dropna(subset=["Clone", "Rep", "t_hr", "VCD"])
      .query(f"{EXP_START} <= t_hr <= {EXP_END}")
      .sort_values(["Clone", "Rep", "t_hr"], ignore_index=True)
)

# ───── 2) Unit conversions (per-mL basis) ───────────────────────────── #
df["Glc_mol_mL"] = df["Glc_g_L"] / MM_GLUCOSE * 1e3 * 1e-6
df["Lac_mol_mL"] = df["Lac_g_L"] / MM_LACTATE  * 1e3 * 1e-6
df["Gln_mol_mL"] = df["Gln_mM"] * 1e-6
df["Glu_mol_mL"] = df["Glu_mM"] * 1e-6
df["rP_g_mL"]    = df["rP_mg_L"] * 1e-6      # mg·L⁻¹ → g·mL⁻¹

# ───── 3) Two-point kinetics per Clone × Rep ────────────────────────── #
records = []

for (clone, rep), g in df.groupby(["Clone", "Rep"], observed=True, sort=False):
    if g.shape[0] < 2:                 # need at least t₀ & t₁
        continue

    t0, t1 = g.iloc[0], g.iloc[-1]
    Δt = t1["t_hr"] - t0["t_hr"]
    if Δt <= 0:
        continue

    # ── Growth & integrals ────────────────────────────────────────── #
    mu = (np.log(t1["VCD"]) - np.log(t0["VCD"])) / Δt

    IVCD_int = ((t0["VCD"] + t1["VCD"]) / 2) * Δt                       # cells·h·mL⁻¹
    IVCC_int = IVCD_int * ((t0["Vol_mL"] + t1["Vol_mL"]) / 2)           # cells·h

    # ── Balances (total content) ──────────────────────────────────── #
    dX   = t1["VCD"] * t1["Vol_mL"] - t0["VCD"] * t0["Vol_mL"]
    dG   = t0["Glc_mol_mL"] * t0["Vol_mL"] - t1["Glc_mol_mL"] * t1["Vol_mL"]
    dL   = t1["Lac_mol_mL"] * t1["Vol_mL"] - t0["Lac_mol_mL"] * t0["Vol_mL"]
    dQln = t0["Gln_mol_mL"] * t0["Vol_mL"] - t1["Gln_mol_mL"] * t1["Vol_mL"]
    dQlu = t1["Glu_mol_mL"] * t1["Vol_mL"] - t0["Glu_mol_mL"] * t0["Vol_mL"]
    dP   = t1["rP_g_mL"]  * t1["Vol_mL"] - t0["rP_g_mL"]  * t0["Vol_mL"]

    # ── Yields ───────────────────────────────────────────────────── #
    Y_XG   = dX / dG   if dG   else np.nan
    Y_XL   = dX / dL   if dL   else np.nan
    Y_XQln = dX / dQln if dQln else np.nan
    Y_XQlu = dX / dQlu if dQlu else np.nan

    # ── Specific rates (pmol o pg·cell⁻¹·h⁻¹) ────────────────────── #
    q_G   = (dG   * 1e12)   / IVCC_int if IVCC_int else np.nan
    q_L   = (dL   * 1e12)   / IVCC_int if IVCC_int else np.nan
    q_Gln = (dQln * 1e12)   / IVCC_int if IVCC_int else np.nan
    q_Glu = (dQlu * 1e12)   / IVCC_int if IVCC_int else np.nan
    q_rP  = (dP   * PG_PER_G) / IVCC_int if IVCC_int else np.nan

    records.append({
        "Clone": clone, "Rep": rep,
        "mu": mu,
        "IVCD": IVCD_int,
        "IVCC": IVCC_int,
        "dX": dX, "dG": dG, "dL": dL, "dQln": dQln, "dQlu": dQlu, "dP": dP,
        "Y_XG": Y_XG, "Y_XL": Y_XL, "Y_XQln": Y_XQln, "Y_XQlu": Y_XQlu,
        "q_G": q_G, "q_L": q_L, "q_Gln": q_Gln, "q_Glu": q_Glu, "q_rP": q_rP
    })

kin_df = pd.DataFrame.from_records(records)

# ───── 4) Save outputs ─────────────────────────────────────────────── #
OUT_REP.parent.mkdir(parents=True, exist_ok=True)

kin_df.to_csv(OUT_REP, index=False)
print(f"✓ Saved Clone × Rep kinetics → {OUT_REP}")

agg = (
    kin_df.groupby("Clone", observed=True)
          .agg(["mean", "std"])
          .reset_index()
)
agg.columns = ["Clone"] + [f"{v}_{s}" for v, s in agg.columns[1:]]
agg.to_csv(OUT_CLONE, index=False)
print(f"✓ Saved Clone-level summary  → {OUT_CLONE}")

# ───── CLI summary ─────────────────────────────────────────────────── #
if __name__ == "__main__":
    print("\n=== Exponential-phase kinetics ===")
    print(f"Window : {EXP_START} – {EXP_END} h")
    print(f"Clones : {kin_df['Clone'].nunique()}")
    print(f"Rows   : {kin_df.shape[0]}")


✓ Saved Clone × Rep kinetics → outputs\kinetics_by_clone_rep.csv
✓ Saved Clone-level summary  → outputs\kinetics_by_clone.csv

=== Exponential-phase kinetics ===
Window : 24.0 – 96.0 h
Clones : 3
Rows   : 9


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
plot_exp.py
~~~~~~~~~~~
Bar-chart visualisation of **clone-level kinetic / stoichiometric metrics**
for a user-defined batch window (e.g. the exponential phase) of CHO
fed-batch cultures.

What it does
------------
* Loads the summary exported by **`exp_phase_kinetics.py`**
  (`outputs/kinetics_by_clone.csv`).
* For every metric in `METRICS` it finds the `<metric>_mean`
  and `<metric>_std` columns, draws a bar plot (mean ± SD) coloured by Clone,
  and saves it as a PNG.

Input
-----
`outputs/kinetics_by_clone.csv`  
(required columns: `Clone` plus `{metric}_mean` and `{metric}_std`
for each metric listed below)

Metrics displayed
-----------------
μ, **IVCD**, **IVCC**, q-G, q-L, q-Gln, q-Glu, q-rP,  
Yₓ/ₛ for Glucose, Lactate, Glutamine, Glutamate  
(extend `METRICS` if you add new variables).

Output
------
PNG files in `outputs/figures_exp/`
(one file per metric, named `<metric>_bar.png`)

Author
------
Emiliano Balderas R. | 16 Jul 2025  
Last edit : 08 Aug 2025 — added IVCC & updated labels
"""

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ───── Configuration ──────────────────────────────────────────────── #
CSV_PATH   = Path("outputs/kinetics_by_clone.csv")
FIGURE_DIR = Path("outputs/figures_exp")
FIGURE_DIR.mkdir(parents=True, exist_ok=True)

FIGSIZE, DPI = (8, 6), 300
AXES_RECT    = [0.15, 0.15, 0.78, 0.78]

sns.set_style("whitegrid")

# ───── Load clone-level summary ────────────────────────────────────── #
if not CSV_PATH.exists():
    raise FileNotFoundError(
        f"❌ File not found:\n  {CSV_PATH}\n"
        "Run `exp_phase_kinetics.py` first."
    )
df = pd.read_csv(CSV_PATH)

# ───── Clone palette ──────────────────────────────────────────────── #
clones = df["Clone"].tolist()
COLORS = dict(zip(clones, sns.color_palette("tab10", len(clones))))

# ───── Helper: bar plot with error bars ───────────────────────────── #
def plot_bar(metric: str, ylabel: str) -> None:
    mean_col, sd_col = f"{metric}_mean", f"{metric}_std"
    means, stds      = df[mean_col].values, df[sd_col].values

    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax  = fig.add_axes(AXES_RECT)

    ax.bar(
        clones, means, yerr=stds,
        capsize=6, edgecolor="black", linewidth=0.8,
        color=[COLORS[c] for c in clones],
        error_kw=dict(ecolor="black", linewidth=1.2)
    )
    ax.set_xlabel("Clone", fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_title(metric, fontsize=13)
    ax.grid(axis="y", linestyle="--", alpha=0.4)

    fig.savefig(FIGURE_DIR / f"{metric}_bar.png")
    plt.close(fig)

# ───── Metrics to plot ────────────────────────────────────────────── #
METRICS = [
    # Growth & integrals
    ("mu",   r"μ (h$^{-1}$)"),
    ("IVCD", r"IVCD (cells·h·mL$^{-1}$)"),
    ("IVCC", r"IVCC (cells·h)"),

    # Specific rates
    ("q_G",   r"q$_{Glc}$ (pmol·cell$^{-1}$·h$^{-1}$)"),
    ("q_L",   r"q$_{Lac}$ (pmol·cell$^{-1}$·h$^{-1}$)"),
    ("q_Gln", r"q$_{Gln}$ (pmol·cell$^{-1}$·h$^{-1}$)"),
    ("q_Glu", r"q$_{Glu}$ (pmol·cell$^{-1}$·h$^{-1}$)"),
    ("q_rP",  r"q$_{rP}$ (pg·cell$^{-1}$·h$^{-1}$)"),

    # Yields
    ("Y_XG",   r"Y$_{X/G}$ (cells·mol$^{-1}$)"),
    ("Y_XL",   r"Y$_{X/L}$ (cells·mol$^{-1}$)"),
    ("Y_XQln", r"Y$_{X/Gln}$ (cells·mol$^{-1}$)"),
    ("Y_XQlu", r"Y$_{X/Glu}$ (cells·mol$^{-1}$)"),
]

for metric, label in METRICS:
    if f"{metric}_mean" in df.columns:
        plot_bar(metric, label)

print("✓ Bar plots saved to ./outputs/figures_exp/")
